In [0]:
!pip install tensorflow==1.15.0

In [0]:
from google.colab import files
files.upload()
# Copy file to the right directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Change permissions owner can read and write
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [0]:
# download the dataset
!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification

In [0]:
!unzip test_public_expanded.csv.zip
!unzip train.csv.zip

In [0]:
import pandas as pd
import numpy as np
from keras.preprocessing import text, sequence
from gensim.models import KeyedVectors

In [0]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test_public_expanded.csv")

In [0]:
print("Train shape: ", train_df.shape, "\nTest shape: ", test_df.shape)

In [0]:
train_df.head(3)

In [0]:
train_df.columns

In [0]:
train_df.describe()

In [0]:
train_df.drop(['created_date', 'publication_id', 'parent_id', 'article_id',
               'rating', 'funny', 'wow', 'sad', 'likes', 'disagree',
               'identity_annotator_count', 'toxicity_annotator_count'], axis=1, inplace=True)

In [0]:
test_df.head(3)

,id,comment_text,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,toxicity,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,threat,identity_annotator_count,toxicity_annotator_count,male,female,transgender,other_gender,heterosexual,homosexual_gay_or_lesbian,bisexual,other_sexual_orientation,christian,jewish,muslim,hindu,buddhist,atheist,other_religion,black,white,asian,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability
0,7000000,Jeff Sessions is another one of Trump's Orwell...,2017-01-26 07:37:38.422417+00,13,NaN,164149,approved,0,0,0,4,0,0.2,0.0,0.0,0.0,0.0,0.2,0.0,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7000001,I actually inspected the infrastructure on Gra...,2016-12-03 20:38:21.204649+00,54,655260.0,154341,approved,0,0,0,2,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7000002,No it won't . That's just wishful thinking on ...,2017-05-05 18:07:58.560078+00,21,5219683.0,332005,approved,0,0,0,0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
test_df.drop(['created_date', 'publication_id', 'parent_id', 'article_id',
               'rating', 'funny', 'wow', 'sad', 'likes', 'disagree',
               'identity_annotator_count', 'toxicity_annotator_count'], axis=1, inplace=True)

In [0]:
test_df.head(3)

In [0]:
print("Average comment_text length is ", np.average([len(i) for i in train_df.comment_text]))

Average comment_text length is  297.2343266067327


In [0]:
MAX_LEN = 300
TEXT_COLUMN = 'comment_text'
TARGET_COLUMN = 'target'
IDENTITY_COLUMNS = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness'
]
AUX_COLUMNS = ['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']

In [0]:
# !pip install emoji
# import emoji
CHARS_TO_REMOVE = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—' #+ "".join([i for i in emoji.UNICODE_EMOJI])

In [0]:
x_train = train_df[TEXT_COLUMN].astype(str)
y_train = train_df[TARGET_COLUMN].values

In [0]:
y_aux_train = train_df[AUX_COLUMNS].values

In [0]:
lstm_tokenizer = text.Tokenizer(filters=CHARS_TO_REMOVE, lower=False)
lstm_tokenizer.fit_on_texts(x_train)

In [0]:
# vectorize a text corpus
x_train = lstm_tokenizer.texts_to_sequences(x_train)
x_train = sequence.pad_sequences(x_train, maxlen=MAX_LEN)

In [0]:
for column in IDENTITY_COLUMNS + [TARGET_COLUMN]:
    train_df[column] = np.where(train_df[column] >= 0.5, True, False)

In [0]:
!kaggle datasets download -d iezepov/gensim-embeddings-dataset -f glove.840B.300d.gensim.vectors.npy
!kaggle datasets download -d iezepov/gensim-embeddings-dataset -f glove.840B.300d.gensim

In [0]:
!unzip glove.840B.300d.gensim.vectors.npy.zip
!unzip glove.840B.300d.gensim.zip

In [0]:
def build_matrix(word_index, path):
    embedding_index = KeyedVectors.load(path, mmap='r')
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    for word, i in word_index.items():
        for candidate in [word, word.lower()]:
          if candidate in embedding_index:
              embedding_matrix[i] = embedding_index[candidate]
              break
    return embedding_matrix

In [0]:
lstm_tokenizer.word_index

In [0]:
embedding_layer_init_weights = build_matrix(lstm_tokenizer.word_index, "glove.840B.300d.gensim")

In [0]:
from keras.layers import Embedding
from keras.layers import Input
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense
from keras.optimizers import RMSprop
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.preprocessing import text, sequence
from gensim.models import KeyedVectors

In [0]:
BATCH_SIZE = 512
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS

In [0]:
def build_model(embedding_matrix, num_aux_targets):
    words = Input(shape=(None,))
    x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)
    hidden = concatenate([
        GlobalMaxPooling1D()(x),
        GlobalAveragePooling1D()(x),
    ])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    result = Dense(1, activation='sigmoid')(hidden)
    aux_result = Dense(num_aux_targets, activation='sigmoid')(hidden)
    model = Model(inputs=words, outputs=[result, aux_result])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
    return model

In [0]:
x_test = test_df[TEXT_COLUMN].astype(str)
y_test = test_df["toxicity"].values

# vectorize TEST text corpus
x_test = lstm_tokenizer.texts_to_sequences(x_test)
x_test = sequence.pad_sequences(x_test, maxlen=MAX_LEN)

In [0]:
model = build_model(embedding_layer_init_weights, y_aux_train.shape[-1])

In [0]:
EPOCHS=2
BATCH_SIZE=512

In [0]:
checkpoint_predictions = []
weights = []

model = build_model(embedding_layer_init_weights, y_aux_train.shape[-1])
for global_epoch in range(EPOCHS):
    model.fit(
        x_train,
        [y_train, y_aux_train],
        batch_size=BATCH_SIZE,
        epochs=1
    )
    checkpoint_predictions.append(model.predict(x_test, batch_size=2048)[0].flatten())
    weights.append(2 ** global_epoch)

In [0]:
MODEL_NAME = 'lstm'
predictions = np.average(checkpoint_predictions, weights=weights, axis=0)

In [0]:
test_df[MODEL_NAME] = predictions

In [0]:
SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

In [0]:
def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

In [0]:
def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]]
    return compute_auc(subgroup_examples[label], subgroup_examples[model_name])

In [0]:
def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[df[subgroup] & ~df[label]]
    non_subgroup_positive_examples = df[~df[subgroup] & df[label]]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label], examples[model_name])

In [0]:
def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[df[subgroup] & df[label]]
    non_subgroup_negative_examples = df[~df[subgroup] & ~df[label]]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label], examples[model_name])

In [0]:
def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)

In [0]:
def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

In [0]:
def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

In [0]:
def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)

In [0]:
# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
TOXICITY_COLUMN = 'toxicity'
TEXT_COLUMN = 'comment_text'

In [0]:
test_df.columns

In [0]:
test_df.lstm

In [0]:
from sklearn import metrics
from sklearn import model_selection

In [0]:
def convert_to_bool(df, col_name):
    df[col_name] = np.where(df[col_name] >= 0.5, True, False)
    
def convert_dataframe_to_bool(df):
    bool_df = df.copy()
    for col in ['toxicity'] + identity_columns:
        convert_to_bool(bool_df, col)
    return bool_df

In [0]:
test_df = convert_dataframe_to_bool(test_df)

In [0]:
bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)

In [0]:
get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, MODEL_NAME))

0.9267225173026998